# LeNet-5 in Tensorflow

#### Load Dependencies

In [ ]:
import numpy as np
import tensorflow as tf

#### Initialise Seeds

In [ ]:
np.random.seed(42)
tf.set_random_seed(42)

#### Load Data

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

In [ ]:
epochs = 10
batch_size = 128
wt_init = tf.contrib.layers.xavier_initializer()
display_progress = 40

#### Set Parameneter for each Layer

In [ ]:
# Input layer:
n_input = 784

# First conv. layer:
n_conv_1 = 64
k_conv_1 = 3

# Scond conv. layer:
n_conv_2 = 64
k_conv_2 = 3

# Max-Pooling layer:
pool_size = 2
mp_layer_dropout = 0.25

# Dense layer:
n_dense = 128
dense_layer_dropout = 0.5

# Output layer:
n_classes = 10

#### Define Placeholders

In [ ]:
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

#### Activation Function

In [ ]:
def relu(z):
    return tf.nn.relu(z)

#### Define Feedforward Computation

In [ ]:
def dense_layer(x, W, b):
    z = tf.add(tf.matmul(x, W), b)
    return relu(z)

def conv2d_layer(x, W, b, stride_length = 1):
    conv_W = tf.nn.conv2d(x, W, strides = [1, stride_length, stride_length, 1], padding = "SAME")
    z = tf.nn.bias_add(conv_W, b)
    return relu(z)

def maxpooling2d_layer(x, p_size):
    return tf.nn.max_pool(x,
                         ksize = [1, p_size, p_size, 1],
                         strides = [1, p_size, p_size, 1],
                         padding = "SAME")

#### Define Neural Network Architecture

In [ ]:
def compute_feedforward(x, weights, biases, n_in, mp_psize, mp_dropout, dense_dropout):
    # Reshape input data:
    square_dimensions = int(np.sqrt(n_in))
    square_x = tf.reshape(x, shape = [-1, square_dimensions, square_dimensions, 1])
    
    # Convolutional layers:
    conv_1 = conv2d_layer(square_x, weights["W_c1"], biases["b_c1"])
    conv_2 = conv2d_layer(conv_1, weights["W_c2"], biases["b_c2"])
    pool_1 = maxpooling2d_layer(conv_2, mp_psize)
    pool_1 = tf.nn.dropout(pool_1, 1 - mp_dropout)
    
    # Dense (fully connected) layer:
    flat = tf.reshape(pool_1, shape = [-1, weights['W_d1'].get_shape().as_list()[0]])
    dense_1 = dense_layer(flat, weights["W_d1"], biases["b_d1"])
    dense_1 = tf.nn.dropout(dense_1, 1 - dense_dropout)
    
    # linear output layer
    out_z = tf.add(tf.matmul(dense_1, weights["W_out"]), biases["b_out"])
    return out_z

#### Build Dictionaries

In [ ]:
bias_dict = {
    "b_c1" : tf.Variable(tf.zeros([n_conv_1])),
    "b_c2" : tf.Variable(tf.zeros([n_conv_2])),
    "b_d1" : tf.Variable(tf.zeros([n_dense])),
    "b_out" : tf.Variable(tf.zeros([n_classes]))
}

full_square_length = np.sqrt(n_input)
pooled_square_length = int(full_square_length / pool_size)
dense_inputs = pooled_square_length ** 2 * n_conv_2

weights_dict = {
    "W_c1" : tf.get_variable("W_c1", [k_conv_1, k_conv_1, 1, n_conv_1], initializer = wt_init),
    "W_c2" : tf.get_variable("W_c2", [k_conv_2, k_conv_2, n_conv_1, n_conv_2], initializer = wt_init),
    "W_d1" : tf.get_variable("W_d1", [dense_inputs, n_dense], initializer = wt_init),
    "W_out" : tf.get_variable("W_out", [n_dense, n_classes], initializer = wt_init)
}

#### Build the Model

In [ ]:
predictions = compute_feedforward(x, weights_dict, bias_dict, n_input, pool_size,
                                  mp_layer_dropout, dense_layer_dropout)

#### Define Loss and Optimizer

In [ ]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = predictions, labels = y))
optimizer = tf.train.AdamOptimizer().minimize(cost)

#### Define  Evaluation Metrics

In [ ]:
correct_predictions = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
accuracy_pct = tf.reduce_mean(tf.cast(correct_predictions, tf.float32)) * 100

#### Initialisation Operation

In [ ]:
initializer_op = tf.global_variables_initializer()

#### Train the Network

In [ ]:
with tf.Session() as session:
    session.run(initializer_op)
    
    print("Training for", epochs, "epochs.")
    
    # loop over epochs:
    for i in range(epochs):
        avg_cost = 0.0
        avg_accuracy_pct = 0.0
        
        # loop over batches:
        n_batches = int(mnist.train.num_examples / batch_size)
        for j in range(n_batches):
            
            # Check progress
            if j % display_progress == 0:
                print("Step ", j + 1, " of ", n_batches, " in epoch ", i + 1, ".", sep = "")
            
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _, batch_cost, batch_acc = session.run([optimizer, cost, accuracy_pct],
                                                   feed_dict = {x : batch_x, y : batch_y})
            avg_cost += batch_cost / n_batches
            avg_accuracy_pct += batch_acc / n_batches
        
        # output logs at end of each epoch of training:\n",
        print("Epoch ", '%03d' % (i + 1),
              ": cost = ", '{:.3f}'.format(avg_cost),
              ", accuracy = ", '{:.2f}'.format(avg_accuracy_pct), "%",
              sep='')

    print("Training Complete. Testing Model.\n")

    test_cost = cost.eval({x: mnist.test.images, y: mnist.test.labels})
    test_accuracy_pct = accuracy_pct.eval({x: mnist.test.images, y: mnist.test.labels})

    print("Test Cost:", '{:.3f}'.format(test_cost))
    print("Test Accuracy:", '{:.2f}'.format(test_accuracy_pct), "%", sep="")